## Automatic Image Captioning for LoRA Dataset Generation

This notebook generates text captions for the preprocessed image dataset
using the BLIP (Bootstrapped Language-Image Pretraining) model. The generated
captions are further enhanced using controlled random variations in viewing
angles, locations, and color descriptions. These captions are saved as
individual `.txt` files and are later used for LoRA and PEFT fine-tuning of
the Stable Diffusion model.

In [9]:
import random
from pathlib import Path
from tqdm import tqdm
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import re

#### PATH CONFIG

In [10]:
IMG_DIR = Path(r"D:\work_space\projects\deep_learning\CAP6415_F25_project-Finding-and-solving-hard-to-generate-examples\data\processed\lora_ready")
DEBUG = True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


#### EXTRA CAPTIONS 

In [11]:
# SAFE VARIATION POOLS
VIEWS = [
    "a close-up of",
    "a detailed view of",
    "a wide view of",
    "a clear view of",
    "a ground-level view of"
]

LOCATIONS = [
    "on an asphalt road surface",
    "on an asphalt street",
    "on a paved roadway",
    "in a parking area on asphalt",
    "on a residential asphalt road"
]

COLORS = [
    "",  # allow some captions without color
    "yellow and black",
    "black and yellow"
]

#### BLIP FOR COLOR HINTS ONLY

In [ ]:
# BLIP 
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(DEVICE)

In [13]:
def extract_color_hint(blip_text):
    blip_text = blip_text.lower()
    if "yellow" in blip_text and "black" in blip_text:
        return random.choice(["yellow and black", "black and yellow"])
    if "yellow" in blip_text:
        return "yellow"
    return ""

#### MAIN CAPTION GENERATER

In [14]:
# CAPTION GENERATOR
def generate_caption(raw_blip):
    view = random.choice(VIEWS)
    location = random.choice(LOCATIONS)
    color = extract_color_hint(raw_blip)

    if color:
        caption = f"{view} {color} speed bump {location}, realistic photo"
    else:
        caption = f"{view} a speed bump {location}, realistic photo"

    caption = re.sub(r"\s+", " ", caption).strip()
    return caption

In [ ]:
# CAPTION LOOP
image_paths = sorted(list(IMG_DIR.glob("*.jpg")))
print(f"\nFound {len(image_paths)} images\n")

count = 0
for img_path in tqdm(image_paths, desc="Safe Varied Captioning"):

    try:
        image = Image.open(img_path).convert("RGB")
        inputs = processor(image, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=40)
        raw_caption = processor.decode(output[0], skip_special_tokens=True)
        final_caption = generate_caption(raw_caption)
        txt_path = img_path.with_suffix(".txt")
        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(final_caption)
        if DEBUG:
            print(f"\nIMAGE: {img_path.name}")
            print(f"BLIP : {raw_caption}")
            print(f"FINAL: {final_caption}")

        count += 1
    except Exception as e:
        print(f"[ERROR] {img_path.name}: {e}")
print(f"\nCaptioning completed for {count} images.")


Found 61 images



Safe Varied Captioning:   2%|▏         | 1/61 [00:00<00:33,  1.77it/s]


IMAGE: speedbump_00000.jpg
BLIP : a yellow traffic light on the side of a road
FINAL: a close-up of yellow speed bump on a residential asphalt road, realistic photo


Safe Varied Captioning:   3%|▎         | 2/61 [00:00<00:27,  2.18it/s]


IMAGE: speedbump_00001.jpg
BLIP : a yellow and black traffic light on the ground
FINAL: a wide view of yellow and black speed bump in a parking area on asphalt, realistic photo


...


IMAGE: speedbump_00058.jpg
BLIP : a man is walking down the street with a yellow tape
FINAL: a detailed view of yellow speed bump on an asphalt street, realistic photo


Safe Varied Captioning:  98%|█████████▊| 60/61 [00:22<00:00,  2.73it/s]


IMAGE: speedbump_00059.jpg
BLIP : a street with a yellow line on the side
FINAL: a close-up of yellow speed bump on a residential asphalt road, realistic photo


Safe Varied Captioning: 100%|██████████| 61/61 [00:23<00:00,  2.65it/s]


IMAGE: speedbump_00060.jpg
BLIP : a person walking down a street with a bike
FINAL: a ground-level view of a speed bump in a parking area on asphalt, realistic photo

Captioning completed for 61 images.


### Code Explanation

This script automatically generates high-quality captions for each image in
the processed LoRA-ready dataset using a pretrained BLIP image-captioning
model. It begins by importing required libraries including PyTorch for GPU
acceleration, PIL for image loading, Hugging Face Transformers for the BLIP
model, and utility libraries such as `random`, `re`, and `tqdm`.

The input image directory is defined using a `Path` object. The script
automatically selects the computing device as CUDA if a GPU is available;
otherwise, it falls back to the CPU. Debug mode is enabled to display
intermediate caption outputs for verification.

To increase caption diversity and reduce overfitting during training, three
controlled variation pools are defined:
- **VIEWS**: Describes camera perspectives.
- **LOCATIONS**: Describes where the speed bump is located.
- **COLORS**: Adds optional color descriptions.

A pretrained BLIP processor and model (`Salesforce/blip-image-captioning-base`)
are loaded from Hugging Face and moved to the selected device for efficient
inference.

The `extract_color_hint` function extracts color cues from the raw BLIP output
to maintain realistic color consistency in the final caption. If a yellow and
black pattern is detected, a randomized color ordering is applied to increase
variation.

The `generate_caption` function constructs the final training caption by
combining a randomized camera view, a randomized road location, and the
derived color hint from the BLIP model. The caption is cleaned using a regular
expression to remove extra spaces and ensure correct formatting.

The script then scans the processed dataset directory for all `.jpg` images
and applies BLIP-based captioning to each image inside a progress-tracked
loop. For every image:
- The image is loaded and converted to RGB.
- The BLIP model generates a raw descriptive caption.
- A controlled final caption is created using the custom generator.
- The caption is saved as a `.txt` file with the same filename as the image.

If debug mode is active, the script prints the image name, raw BLIP caption,
and final generated caption for visual inspection. At the end of the loop, a
summary is printed showing the total number of images successfully captioned.

This automated captioning step produces high-quality, diverse, and
training-ready text labels for effective LoRA + PEFT fine-tuning of the Stable
Diffusion model.